In [1832]:
import numpy as np
from random import random
from functools import reduce
from collections import namedtuple

# constants
PROBLEM_SIZE = 1000  # dimension of the finite set U
NUMBER_SET = 2000  # number of subsets in the collection S
SETS = tuple(
    np.array([random() < 0.3 for i in range(PROBLEM_SIZE)]) for j in range(NUMBER_SET)
)  # generate sets in S
# Define State as a named tuple
State = namedtuple("State", ["taken", "cost", "heuristic"])


def goal_check(state):
    return np.all(
        reduce(np.logical_or, [SETS[i] for i in state.taken], np.zeros(PROBLEM_SIZE))
    )


assert goal_check(State(range(NUMBER_SET), 0, 0))


# Single State

```
Hill-Climbing(problem)
    current <- Make-Node(Initial-State[problem])
    loop do
        neighbor <- a highest-valued successor of current
        if Value[neighbor] <= Value[current] then return State[current]
        current <- neighbor
```


In [1833]:
def covered(taken):
    return np.sum(
        reduce(np.logical_or, [SETS[i] for i in taken], np.zeros(PROBLEM_SIZE))
    )


def overlap(taken):
    # spots that are coveredby just one set in take list

    matrix = np.stack([SETS[i] for i in taken], axis=-1)

    return np.sum(np.sum(matrix, axis=1) > 1)


# heuristic functions
def h1(taken):
    # number of spots covered by the union of the sets in state.taken
    return covered(taken)


def h2(taken):  # doesnmt work with current termination condition s
    # number of spots covered / number of overlapping spots
    c = covered(taken)
    o = overlap(taken)
    return c / o if o else c


In [1834]:
# successor function
def successor(state):
    return [
        State(state.taken + [i], state.cost + 1, h1(state.taken + [i]))
        for i in range(NUMBER_SET)
        if i not in state.taken
    ]


In [1835]:
def hill_climbing(init_state):
    current = init_state
    while True:
        neighbors = successor(current)
        next = max(neighbors, key=lambda state: state.heuristic)

        if (
            goal_check(current)
            or next.heuristic < current.heuristic
            or len(current.taken) == NUMBER_SET
        ):
            return current

        current = next


In [1836]:
# initial state
INITIAL_STATE = State([], 0, 0)

# run hill climbing
res = hill_climbing(INITIAL_STATE)

print("Found   :", res)
print("Is sol  :", goal_check(res))

print("Overlap :", overlap(res.taken))


Found   : State(taken=[967, 1070, 579, 110, 132, 487, 408, 1836, 1008], cost=9, heuristic=1000)
Is sol  : True
Overlap : 862
